# Leaflet cluster map of talk locations

Assuming you are working in a Linux or Windows Subsystem for Linux environment, you may need to install some dependencies. Assuming a clean installation, the following will be needed:

```bash
sudo apt install jupyter
sudo apt install python3-pip
pip install python-frontmatter getorg --upgrade
```

After which you can run this from the `_talks/` directory, via:

```bash
 jupyter nbconvert --to notebook --execute talkmap.ipynb --output talkmap_out.ipynb
```
 
The `_talks/` directory contains `.md` files of all your talks. This scrapes the location YAML field from each `.md` file, geolocates it with `geopy/Nominatim`, and uses the `getorg` library to output data, HTML, and Javascript for a standalone cluster map.

In [1]:
# Start by installing the dependencies
!pip install python-frontmatter getorg --upgrade
import frontmatter
import glob
import getorg
from geopy import Nominatim
from geopy.exc import GeocoderTimedOut

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 6.6 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [getorg]2m7/9 [pygithub]
Iywidgets and ipyleaflet support disabled. You must be in a Jupyter notebook to use this feature.
Error raised:
No module named 'ipyleaflet'
Check that you have enabled ipyleaflet in Jupyter with:
    jupyter nbextension enable --py ipyleaflet


In [5]:
# Collect the Markdown files
g = glob.glob("_talks/*.md")

In [6]:
# Set the default timeout, in seconds
TIMEOUT = 5

# Prepare to geolocate
geocoder = Nominatim(user_agent="academicpages.github.io")
location_dict = {}
location = ""
permalink = ""
title = ""

In [7]:
g

['_talks/2023-10-10-talk-2.md',
 '_talks/2024-06-26-talk-4.md',
 '_talks/2024-06-13-talk-5.md',
 '_talks/2024-06-04-talk-3.md',
 '_talks/2023-09-25-talk-1.md',
 '_talks/2024-03-13-talk-3b.md',
 '_talks/2024-11-06-talk-6.md']

In the event that this times out with an error, double check to make sure that the location is can be properly geolocated.

In [8]:
# Perform geolocation
for file in g:
    # Read the file
    data = frontmatter.load(file)
    data = data.to_dict()

    # Press on if the location is not present
    if 'location' not in data:
        continue

    # Prepare the description
    title = data['title'].strip()
    venue = data['venue'].strip()
    location = data['location'].strip()
    description = f"{title}<br />{venue}; {location}"

    # Geocode the location and report the status
    try:
        location_dict[description] = geocoder.geocode(location, timeout=TIMEOUT)
        print(description, location_dict[description])
    except ValueError as ex:
        print(f"Error: geocode failed on input {location} with message {ex}")
    except GeocoderTimedOut as ex:
        print(f"Error: geocode timed out on input {location} with message {ex}")
    except Exception as ex:
        print(f"An unhandled exception occurred while processing input {location} with message {ex}")

Unveiling Global Patterns in Taxonomic and Gene Expression Dynamics - Species Abundance Distributions and their Biological Interpretation<br />ICTP; Trieste, Italy Trieste, Friuli-Venezia Giulia, 34121-34151, Italia
Complexity and emergence in marine ecosystems/seascape: theory, mechanisms and data analysis<br />Synergy Summer School 2024; Ischia, Italy Ischia, Napoli, Campania, 80077, Italia
Delay effects on the stability of large ecosystems<br />Italian Society of Statistical Physics - Young seminars; Padova, Italy Padova, Veneto, Italia
Deviation from neutral species abundance distributions unveils geographical differences in the structure of diatom communities<br />ICTP; Trieste, Italy Trieste, Friuli-Venezia Giulia, 34121-34151, Italia
Modeling bio-geographies – a conceptual approach<br />AtlantECO and CEODOS; Santiago, Chile Santiago, Provincia de Santiago, Región Metropolitana de Santiago, 8320000, Chile
Deviation from neutral species abundance distributions unveils geographical

In [9]:
# Save the map
m = getorg.orgmap.create_map_obj()
getorg.orgmap.output_html_cluster_map(location_dict, folder_name="talkmap", hashed_usernames=False)

'Written map to talkmap/'